In [ ]:
# 범주형 자료의 수량화 =====================================================
x=[{'city':'seoul','temp':10.0}, {'city':'Dubai', 'temp':33.5}, {'city':'LA','temp':20.0}]
x

In [ ]:
from sklearn.feature_extraction import DictVectorizer
vec=DictVectorizer(sparse=False)
vec.fit_transform(x) # X를 범주형 수량화 자료로 변환

In [ ]:
vec1=DictVectorizer(sparse=True) # 메모리를 줄이기 위해 sparse=True
x1=vec1.fit_transform(x)
x1

In [ ]:
x1.toarray() # 배열로 변경
vec1.get_feature_names()

In [ ]:
# 텍스트 자료의 수량화 =====================================================
text=['떴다 떴다 비행기 날아라 날아라',
      '높이 높이 날아라 우리 비행기',
      '내가 만든 비행기 날아라 날아라',
      '멀리 멀리 날아라 우리 비행기']
text

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec2 = CountVectorizer()
t=vec2.fit_transform(text).toarray() # text를 수량화 배열 자료로 변환
import pandas as pd
t1=pd.DataFrame(t, columns=vec2.get_feature_names())
t1

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfid=TfidfVectorizer()
x2=tfid.fit_transform(text).toarray() # 높은 빈도는 낮은 가중치, 낮은 빈도는 높은 가중치
x3=pd.DataFrame(x2,columns=tfid.get_feature_names())
x3

In [ ]:
# Polynomial 특성변수 생성 =====================================================
import matplotlib.pyplot as plt
import numpy as np
x=np.array([1,2,3,4,5])
y=np.array([5,3,1,5,8])
plt.plot(x,y,'o') # 선형회귀를 하기에는 부적합

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
fg=PolynomialFeatures(degree=3, include_bias=True) # include_bias는 절편항 여부
x1=fg.fit_transform(x[:,np.newaxis]) # 3차까지 생성
print(x1)

In [ ]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(x1,y) 
yfit=reg.predict(x1) # 적합값 
plt.plot(x,y,'ko',label='origin') # ko는 검은색 동그라미
plt.plot(x,yfit,'rs-',label='fitted') # rs-는 빨간색 네모를 줄로 잇기
plt.legend(loc='best') # 범주를 제일 적절한 곳으로

In [ ]:
# 결측자료 대체 ===================================================== 
x_miss=np.array([[1,2,3,None],[5,np.NAN,7,8],[None,10,11,12],[13,np.nan,15,16]])
x_miss

In [ ]:
from sklearn.preprocessing import Imputer
im=Imputer(strategy='mean') 
im.fit_transform(x_miss) # 열의 평균값으로 대체

In [ ]:
# 특성변수 생성(교호작용까지)과 결측자료 대체까지 한번에 해주려면 pipeline 이용.
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import Imputer
y=pd.Series([2,5,1,6])
model = make_pipeline(Imputer(strategy='mean'),PolynomialFeatures(degree=2),LinearRegression())
model.fit(x_miss,y)
model.predict(x_miss) # 이 자료에서는 관측치는 4개, 특성변수 14개로 완전적합됨. 

In [ ]:
#www.kaggle.com 으로 들어간 후 새로운 account를 생성하고 
#www.kaggle.com/c/rossmann-store-sales/data 에서 자료를 다운받는다. 
#zip을 풀어서 적절한 폴더(이경우 머신러닝교재)
import pandas as pd
df1=pd.read_csv("C:/Users/mjsung1023/Desktop/python/train.csv",engine='python')

In [ ]:
df1.shape ## 괄호 없어야함

In [ ]:
type(df1) ## 자주쓰는게 좋다

In [ ]:
df1.head()

In [ ]:
df2=pd.read_csv("C:/Users/mjsung1023/Desktop/python/store.csv",engine='python')
df2.shape

In [ ]:
df2.head()

In [ ]:
df=pd.merge(df1,df2,on='Store')
df.shape

In [ ]:
df.dtypes

In [ ]:
print(len(df['Store'].unique()))
print(len(df['Date'].unique()))
print(df['DayOfWeek'].value_counts())

In [ ]:
import numpy as np
df['Date']=pd.to_datetime(df['Date'],infer_datetime_format=True) ##년.월.일 data를 각각으로 쪼개기 위함
df['Month']=df['Date'].dt.month
df['Quarter']=df['Date'].dt.quarter
df['Year']=df['Date'].dt.year
df['Day']=df['Date'].dt.day
df['Week']=df['Date'].dt.week
df['Season']=np.where(df['Month'].isin([3,4,5]),'Spring',
                     np.where(df['Month'].isin([6,7,8]),'Summer',
                             np.where(df['Month'].isin([9,10,11]),'fall',
                                     np.where(df['Month'].isin([12,1,2]),'winter','None'))))
print(df[['Date','Year','Month','Day','Week','Quarter','Season']].head())

In [ ]:
## np.where(A,a,b) A 이면 a, 아니면 b

In [ ]:
df.hist(figsize=(20,10))

In [ ]:
df.isnull().sum()/df.shape[0]*100

In [ ]:
## 90% 이상이 아니면 버리는게 맞음

In [ ]:
df['CompetitionDistance']=df['CompetitionDistance'].fillna(df['CompetitionDistance'].mode()[0])
df['CompetitionDistance'].isnull().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
n_columns=['Customers','Open','Promo','Promo2','StateHoliday','SchoolHoliday','CompetitionDistance']
categ_columns=['DayOfWeek','Quarter','Month','Year','StoreType','Assortment','Season']

In [ ]:
def dummy(data,col):
   lab=LabelEncoder() #0~c-1로 클래스 부여
   aa=lab.fit_transform(data[col]).reshape(-1,1)
   ohe=OneHotEncoder(sparse=False)
   column_names=[col+'_'+ str(i) for i in lab.classes_]
   return(pd.DataFrame(ohe.fit_transform(aa),columns=column_names))

In [ ]:
fdata=df[n_columns]
for column in categ_columns:
    temp_df=dummy(df,column)
    fdata=pd.concat([fdata,temp_df],axis=1)
fdata.head()

In [ ]:
fdata.shape

In [ ]:
fdata.dtypes.unique()

In [ ]:
fdata['StateHoliday'].unique()

In [ ]:
fdata['StateHoliday']=np.where(fdata['StateHoliday']=='o',0,1)

In [5]:
from sys import executable
print(executable)

C:\Users\mjsung1023\Anaconda3\python.exe


In [6]:
# !pip install imblearn

In [11]:
import matplotlib.pyplot as plt

In [7]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE, ADASYN
X, y = make_classification(n_classes=3, weights=[0.03, 0.07,0.9],n_features=10, 
                           n_clusters_per_class=1, n_samples=2000, random_state=10)
print('Original dataset shape %s' % Counter(y))  

Original dataset shape Counter({2: 1795, 1: 141, 0: 64})


In [8]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({2: 1795, 1: 1795, 0: 1795})


In [9]:
ada=ADASYN(random_state=0)
X_syn,y_syn=ada.fit_resample(X,y)
print('Resampled dataset shape from ADASYN %s' % Counter(y_syn))

Resampled dataset shape from ADASYN Counter({1: 1805, 2: 1795, 0: 1795})
